In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack, hstack
import json
import shutil
import myquickcat
from importlib import reload

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [2]:
full_targets = Table.read("targets/subset_dr8_mtl_dark_gray_NGC.fits")
zcat = Table.read("strategy_A/zcat/dark2_dark3_zcat.fits")
full_mtl = Table.read("strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits")
truth = Table.read("targets/subset_truth_dr8_mtl_dark_gray_NGC.fits")

ii = np.in1d(full_mtl['TARGETID'], zcat['TARGETID'])
in_mtl = full_mtl[ii]

ii = np.in1d(full_targets['TARGETID'], zcat['TARGETID'])
in_targets = full_targets[ii]

print(len(zcat), len(in_mtl), len(in_targets))
in_mtl.sort(keys='TARGETID')
zcat.sort(keys='TARGETID')
in_targets.sort(keys='TARGETID')
truth.sort(keys='TARGETID')


991659 991659 991659


In [3]:
assert np.all(in_mtl['TARGETID']==zcat['TARGETID'])
assert np.all(in_mtl['TARGETID']==in_targets['TARGETID'])

In [16]:
zcat_large = zcat[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_mtl_large = in_mtl[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_targets_large = in_targets[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
print(len(zcat_large), 100*len(zcat_large)/len(zcat))

ii = (zcat['NUMOBS']>3) & (zcat['Z']>2.15)
zcat_large = zcat[ii]
in_mtl_large = in_mtl[ii]
in_targets_large = in_targets[ii]
print(len(zcat_large), 100*len(zcat_large)/len(zcat))

8 0.0008067289259715286
11379 1.1474710560787529


In [17]:
names = {}
name_id = {}
for i in range(len(in_mtl_large)):
    a = desi_mask.names(in_mtl_large['DESI_TARGET'][i])
    n = ''
    for aa in a:
        n = n +' '+ aa
    if n not in names.keys():
        names[n] = 1
    else:
        names[n] += 1
    name_id[in_mtl_large['TARGETID'][i]] = n


In [18]:
names

{' ELG QSO ELG_SOUTH QSO_SOUTH': 3093,
 ' QSO QSO_SOUTH': 6949,
 ' QSO QSO_SOUTH BGS_ANY': 874,
 ' QSO QSO_SOUTH BGS_ANY MWS_ANY': 373,
 ' QSO QSO_SOUTH MWS_ANY': 44,
 ' ELG QSO ELG_SOUTH QSO_SOUTH BGS_ANY': 39,
 ' QSO QSO_SOUTH STD_WD BGS_ANY MWS_ANY': 2,
 ' QSO QSO_SOUTH STD_FAINT BGS_ANY MWS_ANY': 3,
 ' QSO QSO_SOUTH STD_WD MWS_ANY': 1,
 ' LRG QSO LRG_1PASS LRG_SOUTH QSO_SOUTH LRG_1PASS_SOUTH': 1}

In [19]:
zcat_large[:10]

TARGETID,BRICKNAME,SPECTYPE,Z,ZERR,ZWARN,NUMOBS
int64,str8,str10,float32,float32,int32,int32
35185790698917045,,QSO,3.0,0.0,0,4
35185790719889084,,QSO,3.0,0.0,0,4
35185790724083532,,QSO,3.0,0.0,0,4
35185790761833111,,QSO,3.0,0.0,0,4
35185790770220879,,QSO,3.0,0.0,0,4
35185790807968073,,QSO,3.0,0.0,0,4
35185790833134344,,QSO,3.0,0.0,0,4
35185790833135468,,QSO,3.0,0.0,0,4
35185790833135789,,QSO,3.0,0.0,0,4


In [21]:
in_mtl_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
163.891141366,1.29718360295,35185790698917045,393222,0,0,3400,0.240012648499,4,27057,2,3500,3
165.153194815,1.31945779446,35185790719889084,393222,0,0,3400,0.492680522722,4,27038,2,3500,3
165.417995745,1.35643593199,35185790724083532,262148,0,0,3400,0.656004328751,4,27038,2,3500,1
167.568774725,1.36945899338,35185790761833111,393222,0,0,3400,0.900905585941,4,27074,2,3500,3
168.119070507,1.32109565963,35185790770220879,1152921504607109124,262148,0,3400,0.0734089720768,4,27074,2,3500,5
170.301519002,1.24759258702,35185790807968073,393222,0,0,3400,0.920586922237,4,26990,2,3500,3
171.967990436,1.26327763493,35185790833134344,393222,0,0,3400,0.39163500323,4,26989,2,3500,3
171.996406829,1.345539474,35185790833135468,393222,0,0,3400,0.326495971203,4,26989,2,3500,3
171.800675761,1.34972141129,35185790833135789,393222,0,0,3400,0.464994326924,4,26989,2,3500,3


In [22]:
in_targets_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
163.891141366,1.29718360295,35185790698917045,393222,0,0,3400,0.240012648499,4,27057,4,3400,3
165.153194815,1.31945779446,35185790719889084,393222,0,0,3400,0.492680522722,4,27038,4,3400,3
165.417995745,1.35643593199,35185790724083532,262148,0,0,3400,0.656004328751,4,27038,4,3400,1
167.568774725,1.36945899338,35185790761833111,393222,0,0,3400,0.900905585941,4,27074,4,3400,3
168.119070507,1.32109565963,35185790770220879,1152921504607109124,262148,0,3400,0.0734089720768,4,27074,4,3400,5
170.301519002,1.24759258702,35185790807968073,393222,0,0,3400,0.920586922237,4,26990,4,3400,3
171.967990436,1.26327763493,35185790833134344,393222,0,0,3400,0.39163500323,4,26989,4,3400,3
171.996406829,1.345539474,35185790833135468,393222,0,0,3400,0.326495971203,4,26989,4,3400,3
171.800675761,1.34972141129,35185790833135789,393222,0,0,3400,0.464994326924,4,26989,4,3400,3


In [23]:
name_id

{35185790698917045: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790719889084: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790724083532: ' QSO QSO_SOUTH',
 35185790761833111: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790770220879: ' QSO QSO_SOUTH BGS_ANY',
 35185790807968073: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790833134344: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790833135468: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790833135789: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790841523545: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790862495029: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790908632515: ' QSO QSO_SOUTH BGS_ANY MWS_ANY',
 35185790908632659: ' QSO QSO_SOUTH',
 35185790908633105: ' QSO QSO_SOUTH BGS_ANY',
 35185790942185089: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790950575689: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185790954769255: ' QSO QSO_SOUTH',
 35185796692574325: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796700966712: ' QSO QSO_SOUTH',
 35185796709353509: ' ELG QSO ELG_SOUTH QSO_SOUTH',
 35185796717741051: ' ELG 

In [24]:
def consolidate_fassigned(fba_path):
    fba_files = glob.glob(os.path.join(fba_path, 'tile-*fits'))

    # getting all the targetids of the assigned fibers
    print('reading individual fiberassign files')
    favail = list()
    for i_tile, tile_file in enumerate(fba_files):
        if i_tile%50 ==0:
            print(i_tile)
        id_favail, header = fits.getdata(tile_file, 'FIBERASSIGN', header=True)
        favail.extend(id_favail['TARGETID'])
    return np.array(favail)

In [25]:
mtl_names = {0:"strategy_A/targets/gray_subset_dr8_mtl_dark_gray_northern_cap.fits",
             1:"strategy_A/targets/dark0_subset_dr8_mtl_dark_gray_northern_cap.fits",
             2:"strategy_A/targets/dark1_subset_dr8_mtl_dark_gray_northern_cap.fits",
             3:"strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits"}
zcat_names = {0:"strategy_A/zcat/gray_zcat.fits",
             1:"strategy_A/zcat/dark0_zcat.fits",
             2:"strategy_A/zcat/dark1_zcat.fits",
             3:"strategy_A/zcat/dark2_dark3_zcat.fits"}
tile_names = {0:'strategy_A/fiberassign_gray/', 
             1:'strategy_A/fiberassign_dark0/',
             2:'strategy_A/fiberassign_dark1/',
             3:'strategy_A/fiberassign_dark2_dark3/'}

mtl_data = {}
zcat_data = {}
tile_data = {}
tile_files = {}
for k in mtl_names.keys():
    mtl_data[k] = Table.read(mtl_names[k])
    zcat_data[k] = Table.read(zcat_names[k])
    tile_data[k] = consolidate_fassigned(tile_names[k])
    tile_files[k] = glob.glob(os.path.join(tile_names[k], 'tile-*fits'))


reading individual fiberassign files
0
reading individual fiberassign files
0
reading individual fiberassign files
0
50
reading individual fiberassign files
0
50


In [26]:
def print_stats(id_to_track):
    print("ID_TO_TRACK", id_to_track)
    for k in range(4):
        ii = mtl_data[k]['TARGETID']==id_to_track
        jj = zcat_data[k]['TARGETID']==id_to_track
        kk = np.count_nonzero(tile_data[k]==id_to_track)
        fa = Table([[kk]], names=(['NFIBER']))
        if(np.count_nonzero(jj)):
            a = hstack((mtl_data[k][['PRIORITY', 'NUMOBS_MORE']][ii], fa, zcat_data[k][['NUMOBS']][jj]))
        else:
            a = hstack((mtl_data[k][['PRIORITY', 'NUMOBS_MORE']][ii], fa))      
        print(a)
    print()

In [30]:
#ids_to_track = [35185796730325505, 35186182941835863, 35186188885166096, 35186188750949832, 35186194501340726, 35186107519861231]
ids_to_track = [35185827130641806, 35185893358698744, 35186054600327316 , 35186107452755511, 35186165573224800]
ids_to_track = [35185790698917045, 35185833006861227, 35185833027834038, 35185821090841787]

for id_to_track in ids_to_track:
    print_stats(id_to_track)

ID_TO_TRACK 35185790698917045
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           3      0      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           3      1      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           2      2      4

ID_TO_TRACK 35185833006861227
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           3      1      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           2      2      4

ID_TO_TRACK 35185833027834038


PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           3      1      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           2      0      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           2      2      4

ID_TO_TRACK 35185821090841787
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           3      1      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3500           2      2      4



In [99]:
myquickcat = reload(myquickcat)
new_zcat = myquickcat.quickcat(tile_files[3], mtl_data[3], truth, zcat=zcat_data[2], perfect=True)

INFO:myquickcat.py:585:quickcat: Sat Sep 14 13:01:07 2019 QC Reading 98 tiles
INFO:myquickcat.py:608:quickcat: Sat Sep 14 13:01:09 2019 QC Read 98 tiles
INFO:myquickcat.py:630:quickcat: Sat Sep 14 13:01:09 2019 QC Trimming truth to just observed targets
INFO:myquickcat.py:637:quickcat: Sat Sep 14 13:01:10 2019 QC Constructing new redshift catalog
INFO:myquickcat.py:649:quickcat: Sat Sep 14 13:01:10 2019 QC Adding ZERR and ZWARN
INFO:myquickcat.py:667:quickcat: Sat Sep 14 13:01:10 2019 QC Adding NUMOBS column
nobs 1
1. newzcat NUMOBS NUMOBS
------
     1
1. zcat NUMOBS NUMOBS
------
     3
INFO:myquickcat.py:680:quickcat: Sat Sep 14 13:01:12 2019 QC Merging previous zcat
2. newzcat NUMOBS NUMOBS
------
     1
2. zcat NUMOBS NUMOBS
------
     3
3. newzcat NUMOBS NUMOBS
------
     4
3. zcat NUMOBS NUMOBS
------
     4
newzcat NUMOBS NUMOBS
------
     4
INFO:myquickcat.py:738:quickcat: Sat Sep 14 13:01:14 2019 QC done


In [57]:
id_to_track = ids_to_track[-4]
pass_id = 3
print('ID_TO_TRACK', id_to_track)

ll = zcat_data[pass_id-1]['TARGETID']==id_to_track
ii = mtl_data[pass_id]['TARGETID']==id_to_track
jj = np.count_nonzero(tile_data[pass_id]==id_to_track)
kk= new_zcat['TARGETID']==id_to_track
fa = Table([[jj]], names=(['NFIBER']))

print(zcat_data[pass_id-1]['NUMOBS'][ll])
print(mtl_data[pass_id]['NUMOBS_MORE'][ii])
print(fa)
print(new_zcat['NUMOBS'][kk])



ID_TO_TRACK 35186188885166096
NUMOBS
------
     3
NUMOBS_MORE
-----------
          1
NFIBER
------
     1
NUMOBS
------
     5
